In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time # for sleep time
import csv
import re
import functools
import datetime #for csv file title

driver = webdriver.Chrome(r'/Users/msellamia/airbnb/chromedriver')

# Tell Selenium to get the URL you're interested in.
driver.get('https://ca.hotels.com/search.do?resolved-location=CITY%3A1636865%3AUNKNOWN%3AUNKNOWN&destination-id=1636865&q-destination=Toronto,%20Ontario,%20Canada&q-rooms=1&q-room-0-adults=1&q-room-0-children=0')

def reduce_concat(x, sep=""):
    return functools.reduce(lambda x, y: str(x) + sep + str(y), x)

def paste(*lists, sep=" ", collapse=None):
    result = map(lambda x: reduce_concat(x, sep=sep), zip(*lists))
    if collapse is not None:
        return reduce_concat(result, sep=collapse)
    return list(result)

def scroll_bar():
    #scroll to end of page so that all elements are visible
    calm = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//html"))) #if elements aren't available it will return a timeout error and alert user that script must be altered
    time.sleep(3)
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False

    while(match==False):
        lastCount = lenOfPage
        time.sleep(3)
        lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount==lenOfPage:
            match=True

    scroll = driver.find_element_by_xpath('//html')
    time.sleep(2)
    scroll.send_keys(Keys.HOME)
    time.sleep(2)

calm = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//html"))) #if elements aren't available it will return a timeout error and alert user that script must be altered
time.sleep(3)
lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
print(lenOfPage)
wait_event = WebDriverWait(driver, 10)
events = wait_event.until(EC.presence_of_all_elements_located((By.XPATH,'//div[@class="description resp-module"]')))

hotel_all_links = []
newPos = 0
while (newPos <= lenOfPage):
    print('Number of events:', len(events))
    print('position pre-loop:', newPos) #for testing
    scroll_bar()
    time.sleep(3)

    offprep = paste(["var offprep = window.screen.height * "], [(len(events))/5], [";return offprep"], sep='') #7 fit on a page, giving myself room for error
    offprep = ''.join(offprep)
    offset = driver.execute_script(offprep)
    newPos = newPos + offset
    scrollerArg = paste(["window.scrollTo(0, "], [newPos], [");"], sep = '')
    scrollerArg = ''.join(scrollerArg)
    time.sleep(20)
    scroller = driver.execute_script(scrollerArg)
    events = wait_event.until(EC.presence_of_all_elements_located((By.XPATH,'//div[@class="description resp-module"]')))

    print('Number of events:', len(events)) #testing
    print('current position:', newPos) #testing

# Get list of links to hotels and append   
time.sleep(40)
hotel_list = (driver.find_elements_by_xpath('//div[@class="description resp-module"]/figure[@class="image"]/a'))
hotel_all_links = [hotel.get_attribute('href') for hotel in hotel_list ] 

print(len(hotel_all_links))
print('+'*50)
print(hotel_all_links)
time.sleep(4)


time.sleep(1.5)

#Open CSV Writer
now = datetime.datetime.now() #for csv file title
filename = paste(['HotelsManh'], [now.year], ['-'], [now.month], ['-'], [now.day],['.csv'], sep='')
filename = ''.join(filename)
csv_file = open(filename, 'w', encoding='utf-8',newline = '')
writer = csv.writer(csv_file)


#enter the url of hotel after gathering all links and get the variables


for url in hotel_all_links:

    # Initialize an empty dictionary for each review
    #dictionary to store hotel data
    hotel_dict = {}
    try:
       
        driver.get(url)
       
        print('+'*50)
        print(url)
    except:
        continue
    try:
        time.sleep(1.5)
        try:
            hotel_title = driver.find_element_by_xpath('//div[@class="vcard"]/h1').text
        except Exception as e :
            print("issue in hotel title")
            print(e)
            hotel_title = None

        try:
            hotel_type =  driver.find_element_by_xpath('//div[@class="widget-query-group widget-query-destination"]/input').get_attribute('data-as-type')
        except:
            print("issue in hotel type")
            hotel_type = None

        try:

            hotel_price = driver.find_element_by_xpath('//span[@class="current-price"]').text
        except:
            
            try:
                hotel_price = driver.find_element_by_xpath('//div[@class="price"]/span[@class="current-price has-old-price"]').text
                
            except:
                print("issue in hotel_price")    
                hotel_price = None

        try:
            hotel_star = driver.find_element_by_xpath('//span[@class="star-rating-text star-rating-text-strong widget-star-rating-overlay"]').text

        except:
            try:
                hotel_star = driver.find_element_by_xpath('//span[@class="star-rating-text widget-star-rating-overlay"]').text

            except:
                print("issue in hotel star")
                hotel_star = None

        try:
            no_of_ratings = driver.find_element_by_xpath('//div/a[@class="total-reviews trust-you"]').text

        except Exception as e :
            print("issue in no_of_ratings")
            print(e)
            no_of_ratings = None  

        time.sleep(2)        
        try:
            coord_details = driver.find_elements_by_xpath('//span[@class="hotel-coordinates"]/meta')

            lat_long = [detail.get_attribute('content') for detail in coord_details ]

        except:
            print("Issue in lat_long")
            lat_long = None

        time.sleep(2.5)
        try:
            rating = driver.find_element_by_xpath('//div/span[@class="rating"]').text

        except:
            print("issue in rating")
            rating = None
            
        time.sleep(1)
        try:
            occupancy = driver.find_element_by_xpath('//div/span[@class="search-params"]').text

        except:
            print("issue in occupancy")
            occupancy = None
            
        hotel_dict['hotel_title'] = hotel_title
        hotel_dict['hotel_type'] = hotel_type
        hotel_dict['hotel_price'] = hotel_price
        hotel_dict['hotel_star'] = hotel_star
        hotel_dict['no_of_ratings'] = no_of_ratings
        hotel_dict['rating'] = rating
        hotel_dict['occupancy'] = occupancy
        hotel_dict['lat_long'] = lat_long
        
        time.sleep(2)
        print("writing")
        writer.writerow(hotel_dict.values())


        print(hotel_title)
        print(hotel_type)
        print(hotel_price)
        print(hotel_star)
        print(no_of_ratings)
        print(rating)
        print(occupancy)
        print(lat_long)
        
   
    
    except Exception as e:
        print(e)
        continue
        
csv_file.close()
driver.close()

3753
Number of events: 8
position pre-loop: 0
Number of events: 8
current position: 1440
Number of events: 8
position pre-loop: 1440
Number of events: 748
current position: 2880
Number of events: 748
position pre-loop: 2880
Number of events: 748
current position: 137520
748
++++++++++++++++++++++++++++++++++++++++++++++++++
['https://ca.hotels.com/travelads/trackredirect.html?trackingUrl=H4sIAAAAAAAAAD2Q2a6jShZE_-a8UHXIZDBQktXCYAMGMxgw4JcWkJgpGcxo-Pp7brXULxFaEdrSVhTT1I9_SHIa4iXDMRq_6w6h8jvtGtL7m4lodLNhKdOMXBhS7aYMj_-vJFym9X9-jtO6bHM5nuKj1PS_J-Ea6RMaalqs-1B6e4KmLwuJvYZRx_1E-yk23h-LONOiUgn5g7oP4RUiu0l8A3GPpeJ3tWbf4ZI2tkR6ArHIoUtqV_PcHBxor2FFlh_pUvg2IWvhzYyYNDZixZsVeUu4JrBHT_MEqiCivmetYScTu9_13uq3ebFt905gNufYxI6MT5oI2FOUt9ou6HRLxuyp18jZ1_bKRjJTqnbQt5V8IttrHp5mYa7mq2kgX5pa52MZ0eI_hbd616vL-qGkA0k7rg693akF81aAmdP4VjWbKy07Dy7FH_lxLkM1HxGtCbZ6zx2gr4udubGP4oveBjQrM6loyfdC1LKSiheIDwVAeVwgU49uIIC8fXlGdhz7PngunCstaQLga6nq0NT6FsyGy-j6RZJpnrmUfBVXDfVCOwrlGp2vNGe9GJGg461UZBXjCZ30GSvrrIgXGq2wO2RrcEK2w

++++++++++++++++++++++++++++++++++++++++++++++++++
https://ca.hotels.com/travelads/trackredirect.html?trackingUrl=H4sIAAAAAAAAAD2Q2a6jShZE_-a8UHXIZDBQktXCYAMGMxgw4JcWkJgpGcxo-Pp7brXULxFaEdrSVhTT1I9_SHIa4iXDMRq_6w6h8jvtGtL7m4lodLNhKdOMXBhS7aYMj_-vJFym9X9-jtO6bHM5nuKj1PS_J-Ea6RMaalqs-1B6e4KmLwuJvYZRx_1E-yk23h-LONOiUgn5g7oP4RUiu0l8A3GPpeJ3tWbf4ZI2tkR6ArHIoUtqV_PcHBxor2FFlh_pUvg2IWvhzYyYNDZixZsVeUu4JrBHT_MEqiCivmetYScTu9_13uq3ebFt905gNufYxI6MT5oI2FOUt9ou6HRLxuyp18jZ1_bKRjJTqnbQt5V8IttrHp5mYa7mq2kgX5pa52MZ0eI_hbd616vL-qGkA0k7rg693akF81aAmdP4VjWbKy07Dy7FH_lxLkM1HxGtCbZ6zx2gr4udubGP4oveBjQrM6loyfdC1LKSiheIDwVAeVwgU49uIIC8fXlGdhz7PngunCstaQLga6nq0NT6FsyGy-j6RZJpnrmUfBVXDfVCOwrlGp2vNGe9GJGg461UZBXjCZ30GSvrrIgXGq2wO2RrcEK2wmHOMrxMv3uOTny6k1kora_uWZ3QcW99IPjcg2Ha1T2safpcqkWK-oNeE63dORBkcoWNQSIKFK6Xod4Hs0osbsUajfHrrWqIkBnXaLYJt2hUY5p6PaczsOfF88EbjmO9yYEIATMkZvTmMsxJWOzrSEiVj-PRo86xVnNQWbzexXfEtgAcVCsocBBh7fXIqqeTELPfYO3UGgnjgFS6bA987eWm24EI4hkUVtFlZc5LhRdDtstVhtv1AU26-C6qHDv-ekNryKzP7sXSPDI1LbXk_j

++++++++++++++++++++++++++++++++++++++++++++++++++
https://ca.hotels.com/ho115206/?pa=12&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=1
issue in hotel star
writing
Courtyard by Marriott Downtown Toronto, Toronto
HOTEL
$170 CAD
None
See all 1,746 Hotels.com reviews
Very Good8.0
1 room, 1 adult
['43.662218', '-79.382537']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://ca.hotels.com/ho125992/?pa=13&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=1
issue in hotel star
writing
Sheraton Gateway Hotel In Toronto International Airport, Mississauga
HOTEL
$188 CAD
None
See all 1,703 Hotels.com reviews
Good7.6
1 room, 1 adult
['43.686334', '-79.620642']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://ca.hotels.com/ho133487/?pa=14&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=1
issue in hotel star
writing
InterContinental Toronto Centre, Toronto
HOTEL
$162 CAD
None
See all 1,618 Hotels.com reviews
10.0
1 room, 1 adult
['43

issue in hotel star
writing
Park Inn By Radisson Toronto - Markham, Markham
HOTEL
$89 CAD
None
See all 1,402 Hotels.com reviews
Good7.8
1 room, 1 adult
['43.844195', '-79.365925']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://ca.hotels.com/ho215408/?pa=37&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=1
issue in hotel_price
issue in hotel star
writing
Studio 6 Toronto, Toronto
HOTEL
None
None
See all 158 Hotels.com reviews
Very Good8.0
1 room, 1 adult
['43.650994', '-79.400833']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://ca.hotels.com/ho117064/?pa=38&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=1
issue in hotel star
writing
Novotel Toronto Centre, Toronto
HOTEL
$170 CAD
None
See all 1,601 Hotels.com reviews
10.0
1 room, 1 adult
['43.646604', '-79.374696']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://ca.hotels.com/ho143208/?pa=39&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=1
issue in hotel

issue in hotel star
writing
Days Inn by Wyndham Toronto West Mississauga, Mississauga
HOTEL
$114 CAD
None
See all 672 Hotels.com reviews
Good6.4
1 room, 1 adult
['43.626422', '-79.629361']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://ca.hotels.com/ho139614/?pa=62&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=1
issue in hotel_price
issue in hotel star
writing
Park Hyatt Toronto, Toronto
HOTEL
None
None
See all 180 Hotels.com reviews
Good7.0
1 room, 1 adult
['43.668756', '-79.394161']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://ca.hotels.com/ho149121/?pa=63&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=1
issue in hotel star
writing
Holiday Inn Toronto Airport East, Toronto
HOTEL
$103 CAD
None
See all 351 Hotels.com reviews
Good7.2
1 room, 1 adult
['43.691346', '-79.572735']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://ca.hotels.com/ho141624/?pa=64&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults

...

++++++++++++++++++++++++++++++++++++++++++++++++++
https://ca.hotels.com/ho775382752/?pa=673&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=1
issue in hotel star
issue in no_of_ratings
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div/a[@class="total-reviews trust-you"]"}
  (Session info: chrome=78.0.3904.70)

writing
Executive Suite Entertainment District, Toronto
HOTEL
$100 CAD
None
None
Very Good8.0
1 room, 1 adult
['43.647605', '-79.392562']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://ca.hotels.com/ho776070560/?pa=674&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=1
issue in hotel_price
issue in hotel star
issue in no_of_ratings
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div/a[@class="total-reviews trust-you"]"}
  (Session info: chrome=78.0.3904.70)

writing
Style in Yorkville, Toronto
HOTEL
None
None
None
Good7.0
1 room, 1 adult
['43.67216', '-79.39032']


issue in hotel_price
issue in hotel star
issue in no_of_ratings
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div/a[@class="total-reviews trust-you"]"}
  (Session info: chrome=78.0.3904.70)

writing
Execstay - Luxury Yorkville Suites, Toronto
HOTEL
None
None
None
Exceptional9.6
1 room, 1 adult
['43.67037', '-79.39413']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://ca.hotels.com/ho847355840/?pa=690&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=1
issue in hotel_price
issue in hotel star
issue in no_of_ratings
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div/a[@class="total-reviews trust-you"]"}
  (Session info: chrome=78.0.3904.70)

writing
Downtown Studio in Dundas Square, Toronto
HOTEL
None
None
None
Good7.0
1 room, 1 adult
['43.654546', '-79.378829']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://ca.hotels.com/ho851991552/?pa=691&tab=description&ZSX=0&SYE=3&q-room-0-

writing
Cozy Studio Condo with Great View, Toronto
HOTEL
None
None
None
Exceptional9.6
1 room, 1 adult
['43.63712', '-79.40462']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://ca.hotels.com/ho869139648/?pa=706&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=1
issue in hotel_price
issue in hotel star
issue in no_of_ratings
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div/a[@class="total-reviews trust-you"]"}
  (Session info: chrome=78.0.3904.70)

writing
3 Bedrooms Design Home, Toronto
HOTEL
None
None
None
Good7.6
1 room, 1 adult
['43.67271', '-79.40647']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://ca.hotels.com/ho869384672/?pa=707&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=1
issue in hotel_price
issue in hotel star
issue in no_of_ratings
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div/a[@class="total-reviews trust-you"]"}
  (Session info: chrome

issue in rating
writing
Luxury Lakeview Skyscraper by CN Tower, Toronto
HOTEL
None
None
None
None
1 room, 1 adult
['43.641992', '-79.38162']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://ca.hotels.com/ho886250400/?pa=722&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=1
issue in hotel star
issue in no_of_ratings
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div/a[@class="total-reviews trust-you"]"}
  (Session info: chrome=78.0.3904.70)

writing
The Woodbine Beach House, Toronto
HOTEL
$79 CAD
None
None
Good7.0
1 room, 1 adult
['43.66925', '-79.3014']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://ca.hotels.com/ho887994080/?pa=723&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=1
issue in hotel star
issue in no_of_ratings
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div/a[@class="total-reviews trust-you"]"}
  (Session info: chrome=78.0.3904.70)

writing
Th

issue in hotel star
issue in no_of_ratings
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div/a[@class="total-reviews trust-you"]"}
  (Session info: chrome=78.0.3904.70)

writing
The Maple Boutique Suites, Toronto
HOTEL
$140 CAD
None
None
Exceptional9.6
1 room, 1 adult
['43.64291', '-79.38054']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://ca.hotels.com/ho939567136/?pa=739&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=1
issue in hotel_price
issue in hotel star
issue in no_of_ratings
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div/a[@class="total-reviews trust-you"]"}
  (Session info: chrome=78.0.3904.70)

writing
Grand 2BR Condo with Great Views, Toronto
HOTEL
None
None
None
Good7.0
1 room, 1 adult
['43.64761', '-79.39057']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://ca.hotels.com/ho939570624/?pa=740&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=1